In [2]:
%%bash
if command -v uv > /dev/null; then
    uv add --upgrade "dask[dataframe]"
    uv add --upgrade "dask[distributed]"
    uv add pandas
    uv add numpy
    uv add plotly
else
    pip install --upgrade "dask[dataframe]"
    pip install --upgrade "dask[distributed]"
    pip install pandas
    pip install numpy
    pip install plotly
fi

Resolved 59 packages in 426ms
Audited 49 packages in 0.02ms
Resolved 59 packages in 166ms
Audited 49 packages in 0.02ms
Resolved 59 packages in 21ms
Audited 49 packages in 0.02ms
Resolved 59 packages in 18ms
Audited 49 packages in 0.02ms
Resolved 61 packages in 378ms
Prepared 2 packages in 908ms
Installed 2 packages in 92ms
 + narwhals==1.27.1
 + plotly==6.0.0


In [3]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

from dask.distributed import Client, LocalCluster

# Existe integração com pandas, então customização da configuração do pandas também funciona aqui

# Não limitar a largura das colunas apresentadas
pd.options.display.max_colwidth = None
# Não usar a notação científica (ex: 6.125000e-02) e usar 6 casas decimais (ex: 0.061250)
pd.options.display.float_format = "{:.6f}".format
# Não utilizar matplotlib como engine de gráficos e usar plotly
pd.options.plotting.backend = "plotly"

In [4]:
# Criar cluster local anexado ao kernel do notebook
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 31.09 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34457,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 31.09 GiB
Comm: tcp://127.0.0.1:42235,Total threads: 4
Dashboard: http://127.0.0.1:34717/status,Memory: 7.77 GiB
Nanny: tcp://127.0.0.1:33415,


In [5]:
ROOT_DATA_PATH = "/home/mgrb/Workspaces/CESAR.School/big_data/data/ml-25m"

In [6]:
movies_df = dd.read_csv(f"{ROOT_DATA_PATH}/movies.csv")
movies_df

,movieId,title,genres
npartitions=1,,,
,int64,string,string
,...,...,...


In [7]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
len(movies_df)

62423

In [9]:
tags_df = dd.read_csv(f"{ROOT_DATA_PATH}/tags.csv")
tags_df

,userId,movieId,tag,timestamp
npartitions=1,,,,
,int64,int64,string,int64
,...,...,...,...


In [10]:
tags_df.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [11]:
len(tags_df)

1093360

In [12]:
# Converter os valores da coluna 'timestamp' de epoch em segundos para datetime
tags_df['timestamp'] = dd.to_datetime(tags_df['timestamp'], unit='s')
tags_df.head()

,userId,movieId,tag,timestamp
0,3,260,classic,2015-08-13 13:25:55
1,3,260,sci-fi,2015-08-13 13:24:16
2,4,1732,dark comedy,2019-11-16 22:33:18
3,4,1732,great dialogue,2019-11-16 22:33:24
4,4,7569,so bad it's good,2019-11-16 22:30:55


In [13]:
ratings_df = dd.read_csv(f"{ROOT_DATA_PATH}/ratings.csv")
ratings_df

,userId,movieId,rating,timestamp
npartitions=10,,,,
,int64,int64,float64,int64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [14]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.000000,1147880044
1,1,306,3.500000,1147868817
2,1,307,5.000000,1147868828
3,1,665,5.000000,1147878820
4,1,899,3.500000,1147868510


In [15]:
# Converter os valores da coluna 'timestamp' de epoch em segundos para datetime
ratings_df['timestamp'] = dd.to_datetime(ratings_df['timestamp'], unit='s')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.000000,2006-05-17 15:34:04
1,1,306,3.500000,2006-05-17 12:26:57
2,1,307,5.000000,2006-05-17 12:27:08
3,1,665,5.000000,2006-05-17 15:13:40
4,1,899,3.500000,2006-05-17 12:21:50


In [16]:
%%time

len(ratings_df)

CPU times: user 89.1 ms, sys: 12.8 ms, total: 102 ms
Wall time: 1.14 s


25000095

In [17]:
links_df = dd.read_csv(
    f"{ROOT_DATA_PATH}/links.csv",
    dtype={"movieId": int, "imdbId": int},
    assume_missing=True,
)
links_df

,movieId,imdbId,tmdbId
npartitions=1,,,
,int64,int64,float64
,...,...,...


In [18]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.000000
1,2,113497,8844.000000
2,3,113228,15602.000000
3,4,114885,31357.000000
4,5,113041,11862.000000


In [19]:
len(links_df)

62423

In [20]:
links_df.isnull().sum().compute()

movieId      0
imdbId       0
tmdbId     107
dtype: int64

In [21]:
gtags_df = dd.read_csv(f"{ROOT_DATA_PATH}/genome-tags.csv")
gtags_df

,tagId,tag
npartitions=1,,
,int64,string
,...,...


In [22]:
gtags_df.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [23]:
len(gtags_df)

1128

In [24]:
gscores_df = dd.read_csv(f"{ROOT_DATA_PATH}/genome-scores.csv")
gscores_df

,movieId,tagId,relevance
npartitions=6,,,
,int64,int64,float64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [25]:
gscores_df.head()

,movieId,tagId,relevance
0,1,1,0.028750
1,1,2,0.023750
2,1,3,0.062500
3,1,4,0.075750
4,1,5,0.140750


In [26]:
%%time

len(gscores_df)

CPU times: user 40.4 ms, sys: 5.22 ms, total: 45.6 ms
Wall time: 639 ms


15584448

## Executando merde de Dados

In [27]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [28]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.000000
1,2,113497,8844.000000
2,3,113228,15602.000000
3,4,114885,31357.000000
4,5,113041,11862.000000


In [29]:
# Calculando total de linhas
len(movies_df)

62423

In [30]:
len(links_df)

62423

In [31]:
# Criando operação de merge entre filmes e links
merged_df = dd.merge(movies_df, links_df, on='movieId', how='inner')
merged_df

,movieId,title,genres,imdbId,tmdbId
npartitions=1,,,,,
,int64,string,string,int64,float64
,...,...,...,...,...


In [32]:
merged_df.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.000000
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.000000
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.000000
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.000000
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.000000


In [33]:
len(merged_df)

62423

In [34]:
# Uma forma alternativa de fazer merge seria utilizando um dos dataframes
movies_df = movies_df.merge(links_df, on='movieId', how='inner')
movies_df

,movieId,title,genres,imdbId,tmdbId
npartitions=1,,,,,
,int64,string,string,int64,float64
,...,...,...,...,...


In [35]:
movies_df.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.000000
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.000000
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.000000
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.000000
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.000000


In [36]:
len(movies_df)

62423

## Quais são os top 10 fimes mais avaliados?

In [37]:
client.close()
cluster.close()